In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'o3': 'atmpro'}
band = [7]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.66, 1.66, 1.8)]
  wgt = [(0.5, 0.55, 0.9)]
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)]
  wgt = [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.038950                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.030531
0.000750      2.0 -0.043333                  2.0 -0.033993
0.001052      3.0 -0.051784                  3.0 -0.040736
0.001476      4.0 -0.059892                  4.0 -0.047233
0.002070      5.0 -0.067668                  5.0 -0.053504
0.002904      6.0 -0.075126                  6.0 -0.059561
0.004074      7.0 -0.082241                  7.0 -0.065375
0.005714      8.0 -0.089010                  8.0 -0.070952
0.008015      9.0 -0.095439                  9.0 -0.076272
0.011243     10.0 -0.101549                 10.0 -0.081357
0.015771     11.0 -0.107488                 11.0 -0.086184
0.022122     12.0 -0.113271                 12.0 -0.090790
0.031031     13.0 -0.119064                 13.0 -0.095186
0.043528     14.0 -0.124809                 14.0 -0.099255
0.061057     15.0 -0.126774                 15.0 -0.100566
0.085645     16.0 -0.117658                 16.0 -0.094330
0.120136     17.0 -0.069658                 17.0 -0.057572
0.168516     18.0  0.032691                 18.0  0.026138
0.236378     19.0  0.199575                 19.0  0.169488
0.331549     20.0  0.443062                 20.0  0.387820
0.465100     21.0  0.776219                 21.0  0.697328
0.652400     22.0  1.218532                 22.0  1.114065
0.915100     23.0  1.733810                 23.0  1.609966
1.283650     24.0  1.995515                 24.0  1.913312
1.800600     25.0  1.792542                 25.0  1.782842
2.525700     26.0  1.438781                 26.0  1.435428
3.542800     27.0  1.064743                 27.0  1.056236
4.969550     28.0  0.747785                 28.0  0.755830
6.970850     29.0  0.525825                 29.0  0.556839
9.778100     30.0  0.356046                 30.0  0.400786
13.715850    31.0  0.183653                 31.0  0.224464
19.239350    32.0  0.015207                 32.0  0.039186
26.987250    33.0 -0.102961                 33.0 -0.095874
37.855300    34.0 -0.185968                 34.0 -0.194897
53.100050    35.0 -0.255555                 35.0 -0.282250
73.887500    36.0 -0.277647                 36.0 -0.318448
97.662500    37.0 -0.246003                 37.0 -0.286553
121.437500   38.0 -0.213081                 38.0 -0.246330
145.212500   39.0 -0.198159                 39.0 -0.223116
168.987500   40.0 -0.177253                 40.0 -0.193157
192.762500   41.0 -0.141799                 41.0 -0.150826
216.537500   42.0 -0.112201                 42.0 -0.117658
240.312500   43.0 -0.091922                 43.0 -0.095187
264.087500   44.0 -0.073206                 44.0 -0.074880
287.862500   45.0 -0.058414                 45.0 -0.059109
311.637500   46.0 -0.048552                 46.0 -0.048643
335.412500   47.0 -0.040154                 47.0 -0.039832
359.187500   48.0 -0.032376                 48.0 -0.031801
382.962500   49.0 -0.025689                 49.0 -0.024982
406.737500   50.0 -0.019950                 50.0 -0.019193
430.512500   51.0 -0.014679                 51.0 -0.013935
454.287500   52.0 -0.009865                 52.0 -0.009174
478.062500   53.0 -0.005479                 53.0 -0.004870
501.837500   54.0 -0.001509                 54.0 -0.000993
525.612500   55.0  0.002204                 55.0  0.002605
549.387500   56.0  0.005784                 56.0  0.006060
573.162500   57.0  0.009309                 57.0  0.009453
596.937500   58.0  0.013102                 58.0  0.013083
620.712500   59.0  0.017022                 59.0  0.016842
644.487500   60.0  0.021075                 60.0  0.020712
668.262500   61.0  0.025293                 61.0  0.024725
692.037500   62.0  0.029658                 62.0  0.028897
715.812500   63.0  0.034232                 63.0  0.033323
739.587500   64.0  0.039060                 64.0  0.038099
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -22.783978  0.000000 -22.783978        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -22.390740  0.000002   
6.244000e-04 2     -22.783983  0.000001 -22.783981 -22.390744  0.000003   
8.759000e-04 3     -22.783985  0.000002 -22.783983 -22.390745  0.000003   
1.228600e-03 4     -22.783988  0.000003 -22.783985 -22.390748  0.000004   
1.723400e-03 5     -22.783993  0.000005 -22.783988 -22.390752  0.000005   
2.417400e-03 6     -22.784002  0.000008 -22.783994 -22.390758  0.000008   
3.390900e-03 7     -22.784016  0.000013 -22.784002 -22.390769  0.000011   
4.756500e-03 8     -22.784037  0.000021 -22.784016 -22.390785  0.000017   
6.672000e-03 9     -22.784070  0.000034 -22.784036 -22.390810  0.000026   
9.358900e-03 10    -22.784121  0.000054 -22.784066 -22.390849  0.000040   
1.312780e-02 11    -22.784197  0.000086 -22.784112 -22.390907  0.000063   
1.841450e-02 12    -22.784313  0.000134 -22.784179 -22.390996  0.000097   
2.583020e-02 13    -22.784486  0.000207 -22.784278 -22.391128  0.000149   
3.623230e-02 14    -22.784745  0.000320 -22.784425 -22.391326  0.000230   
5.082340e-02 15    -22.785130  0.000490 -22.784641 -22.391620  0.000352   
7.129060e-02 16    -22.785711  0.000763 -22.784948 -22.392063  0.000552   
1.000000e-01 17    -22.786656  0.001308 -22.785348 -22.392785  0.000953   
1.402710e-01 18    -22.788183  0.002503 -22.785681 -22.393956  0.001849   
1.967600e-01 19    -22.790448  0.004986 -22.785462 -22.395688  0.003756   
2.759970e-01 20    -22.793501  0.009912 -22.783589 -22.397988  0.007647   
3.871000e-01 21    -22.797060  0.019303 -22.777758 -22.400544  0.015308   
5.431000e-01 22    -22.799985  0.036572 -22.763414 -22.402264  0.029916   
7.617000e-01 23    -22.799035  0.067175 -22.731860 -22.400264  0.056767   
1.068500e+00 24    -22.787973  0.119124 -22.668849 -22.389200  0.104219   
1.498800e+00 25    -22.762966  0.195833 -22.567133 -22.365557  0.178112   
2.102400e+00 26    -22.727400  0.288435 -22.438965 -22.331752  0.271794   
2.949000e+00 27    -22.683899  0.389223 -22.294676 -22.289893  0.373903   
4.136600e+00 28    -22.637804  0.492916 -22.144888 -22.244988  0.477605   
5.802500e+00 29    -22.597356  0.600034 -21.997322 -22.205127  0.586913   
8.139200e+00 30    -22.570207  0.718433 -21.851774 -22.177668  0.713602   
1.141700e+01 31    -22.567724  0.854195 -21.713529 -22.173733  0.865300   
1.601470e+01 32    -22.612992  0.999485 -21.613506 -22.216547  1.030376   
2.246400e+01 33    -22.739856  1.137967 -21.601889 -22.341016  1.184785   
3.151050e+01 34    -22.983786  1.271562 -21.712224 -22.586794  1.327812   
4.420010e+01 35    -23.399662  1.407896 -21.991766 -23.018818  1.466842   
6.200000e+01 36    -24.058651  1.528040 -22.530611 -23.734034  1.586864   
8.577500e+01 37    -24.924027  1.611473 -23.312553 -24.715547  1.671433   
1.095500e+02 38    -25.667803  1.662428 -24.005375 -25.575099  1.723877   
1.333250e+02 39    -26.308391  1.702911 -24.605480 -26.310380  1.765342   
1.571000e+02 40    -26.900243  1.736684 -25.163560 -26.973704  1.800235   
1.808750e+02 41    -27.431222  1.768461 -25.662761 -27.550135  1.832619   
2.046500e+02 42    -27.868918  1.806805 -26.062113 -28.012120  1.869786   
2.284250e+02 43    -28.234258  1.856151 -26.378108 -28.389884  1.916153   
2.522000e+02 44    -28.551467  1.914478 -26.636989 -28.712405  1.970568   
2.759750e+02 45    -28.820018  1.976858 -26.843159 -28.981453  2.028707   
2.997500e+02 46    -29.049376  2.041704 -27.007672 -29.208367  2.089133   
3.235250e+02 47    -29.255224  2.110816 -27.144408 -29.409796  2.153554   
3.473000e+02 48    -29.441138  2.183644 -27.257494 -29.589907  2.221473   
3.710750e+02 49    -29.607552  2.258876 -27.348676 -29.749656  2.291651   
3.948500e+02 50    -29.757633  2.336608 -27.421024 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')